## Examen Parcial CC0C2

**Nombre y Apellidos: Juan José Camarena Zamalloa**

**Código: 20171342H**

### Reglas para el Examen Parcial

- Queda terminantemente prohibido el uso de herramientas como ChatGPT, WhatsApp, o cualquier herramienta similar durante la realización de esta prueba. El uso de estas herramientas, por cualquier motivo, resultará en la anulación inmediata de la evaluación. Puedes utilizar los cuadernos y datos alojados en github.

- Las respuestas deben presentarse con una explicación detallada, utilizando términos técnicos apropiados. La mera descripción sin el uso de terminología técnica, especialmente términos discutidos en clase, se considerará insuficiente y podrá resultar en que la respuesta sea marcada como incorrecta.

- Cada estudiante debe presentar su propio trabajo. Los códigos iguales o muy parecidos entre sí serán considerados como una violación a la integridad académica, implicando una copia, y serán sancionados de acuerdo con las políticas de la universidad.

- Todos los estudiantes deben subir sus repositorios de código a la plataforma del curso, según las instrucciones proporcionadas. La fecha y hora de la última actualización del repositorio serán consideradas como la hora de entrega.

- La claridad, orden, y presentación general de las evaluaciones serán tomadas en cuenta en la calificación final. Se espera un nivel de profesionalismo en la documentación y presentación del código y las respuestas escritas.


#### Instrucciones de entrega para la prueba calificada

- Presenta la dirección de tu repositorio personal donde se encuentre este cuaderno con tus respuestas desarrolladas.
- Todo cambio fuera de la hora y fecha del examen realizado dentro del repositorio no se tomará en cuenta y se procederá a anular la evaluación.

### Problema 1

El subsampling en el contexto de los modelos de Word2Vec es una técnica utilizada para reducir el número de veces que se entrenan palabras muy frecuentes. Se basa en la idea de que las palabras extremadamente comunes (como preposiciones y conjunciones) proporcionan menos información de contexto valiosa en comparación con las palabras menos frecuentes. En la práctica, cada palabra en el conjunto de entrenamiento tiene una probabilidad calculada de ser "saltada" durante el entrenamiento, dependiendo de su frecuencia. Esto ayuda a acelerar el entrenamiento y a mejorar la calidad de las representaciones de palabras menos frecuentes, que podrían verse oscurecidas por palabras de alta frecuencia.

El negative sampling es una técnica de optimización para reducir la complejidad computacional de actualizar los pesos en la red neuronal en modelos como Word2Vec. En lugar de actualizar los pesos de todas las palabras del vocabulario para cada ejemplo de entrenamiento (lo cual es muy costoso computacionalmente), el negative sampling actualiza solo un pequeño número de "palabras negativas" (ejemplos negativos seleccionados aleatoriamente) junto con la palabra objetivo (ejemplo positivo). Esto no solo acelera significativamente el entrenamiento sino que también mejora la calidad de las representaciones vectoriales al enfocarse en distinguir la palabra objetivo de un pequeño subconjunto de palabras negativas.

La correlación de Spearman es una medida estadística que evalúa la fuerza y la dirección de la asociación entre dos variables clasificadas. A diferencia de la correlación de Pearson, que requiere que las variables sean de escala intervalo o de razón y aproximadamente normales, la correlación de Spearman no hace suposiciones sobre la distribución de los datos y se basa en rangos. Es especialmente útil en el contexto de Word2Vec cuando se evalúa cómo las similitudes coseno calculadas entre vectores de palabras se comparan con juicios humanos de similitud (usualmente dados en estudios donde las personas califican qué tan similares son las palabras). Al correlacionar estos dos conjuntos de rankings (el calculado y el humano), se puede obtener una medida de cuán bien el modelo captura relaciones semánticas que coinciden con las percepciones humanas.

#### Ejercicios:

1. Implementa los modelos CBOW y Skip-gram en Python sin utilizar bibliotecas de alto nivel como Gensim (2 puntos).
    - Escribe el código para inicializar los pesos de la red, realizar el entrenamiento mediante descenso de gradiente y calcular la función de pérdida.
    - Añade mecanismos de subsampling y negative sampling para mejorar la eficiencia del entrenamiento.
2. Analiza cómo diferentes hiperparámetros afectan la calidad de los embeddings vectoriales (2 puntos).
    - Entrena modelos Word2Vec con diferentes tamaños de ventana, dimensiones de vector y tasas de aprendizaje. Utiliza un conjunto de datos estándar como el corpus de texto de Wikipedia.
    - Evalúa los modelos usando tareas de analogía de palabras y calcula la correlación de Spearman entre las similitudes humanas y las  calculadas por el modelo.



Codigo:

In [2]:
import numpy as np
import string
from nltk.corpus import stopwords

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

class word2vec(object):
    def __init__(self):
        self.N = 10
        self.X_train = []
        self.y_train = []
        self.window_size = 2
        self.alpha = 0.001
        self.words = []
        self.word_index = {}

    def initialize(self,V,data):
        self.V = V
        self.W = np.random.uniform(-0.8, 0.8, (self.V, self.N))
        self.W1 = np.random.uniform(-0.8, 0.8, (self.N, self.V))

        self.words = data
        for i in range(len(data)):
            self.word_index[data[i]] = i


    def feed_forward(self,X):
        self.h = np.dot(self.W.T,X).reshape(self.N,1)
        self.u = np.dot(self.W1.T,self.h)
        #print(self.u)
        self.y = softmax(self.u)
        return self.y

    def backpropagate(self,x,t):
        e = self.y - np.asarray(t).reshape(self.V,1)
        # e.shape is V x 1
        dLdW1 = np.dot(self.h,e.T)
        X = np.array(x).reshape(self.V,1)
        dLdW = np.dot(X, np.dot(self.W1,e).T)
        self.W1 = self.W1 - self.alpha*dLdW1
        self.W = self.W - self.alpha*dLdW

    def train(self,epochs):
        for x in range(1,epochs):
            self.loss = 0
            for j in range(len(self.X_train)):
                self.feed_forward(self.X_train[j])
                self.backpropagate(self.X_train[j],self.y_train[j])
                C = 0
                for m in range(self.V):
                    if(self.y_train[j][m]):
                        self.loss += -1*self.u[m][0]
                        C += 1
                self.loss += C*np.log(np.sum(np.exp(self.u)))
            print("epoch ",x, " loss = ",self.loss)
            self.alpha *= 1/( (1+self.alpha*x) )

    def predict(self,word,number_of_predictions):
        if word in self.words:
            index = self.word_index[word]
            X = [0 for i in range(self.V)]
            X[index] = 1
            prediction = self.feed_forward(X)
            output = {}
            for i in range(self.V):
                output[prediction[i][0]] = i

            top_context_words = []
            for k in sorted(output,reverse=True):
                top_context_words.append(self.words[output[k]])
                if(len(top_context_words)>=number_of_predictions):
                    break

            return top_context_words
        else:
            print("Word not found in dictionary")

In [3]:

def preprocessing(corpus):
    stop_words = set(stopwords.words('english'))
    training_data = []
    sentences = corpus.split(".")
    for i in range(len(sentences)):
        sentences[i] = sentences[i].strip()
        sentence = sentences[i].split()
        x = [word.strip(string.punctuation) for word in sentence
                                     if word not in stop_words]
        x = [word.lower() for word in x]
        training_data.append(x)
    return training_data


def prepare_data_for_training(sentences,w2v):
    data = {}
    for sentence in sentences:
        for word in sentence:
            if word not in data:
                data[word] = 1
            else:
                data[word] += 1
    V = len(data)
    data = sorted(list(data.keys()))
    vocab = {}
    for i in range(len(data)):
        vocab[data[i]] = i

    #for i in range(len(words)):
    for sentence in sentences:
        for i in range(len(sentence)):
            center_word = [0 for x in range(V)]
            center_word[vocab[sentence[i]]] = 1
            context = [0 for x in range(V)]

            for j in range(i-w2v.window_size,i+w2v.window_size):
                if i!=j and j>=0 and j<len(sentence):
                    context[vocab[sentence[j]]] += 1
            w2v.X_train.append(center_word)
            w2v.y_train.append(context)
    w2v.initialize(V,data)

    return w2v.X_train,w2v.y_train

In [6]:
corpus = ""
corpus += "The earth revolves around the sun. The moon revolves around the earth"
epochs = 1000

training_data = preprocessing(corpus)
w2v = word2vec()

prepare_data_for_training(training_data,w2v)
w2v.train(epochs)

print(w2v.predict("around",3))

epoch  1  loss =  41.40161488251882
epoch  2  loss =  41.33491393373918
epoch  3  loss =  41.268722969649346
epoch  4  loss =  41.20310167307102
epoch  5  loss =  41.13810730625654
epoch  6  loss =  41.07379441660124
epoch  7  loss =  41.01021457497016
epoch  8  loss =  40.9474161495398
epoch  9  loss =  40.88544411724018
epoch  10  loss =  40.82433991405069
epoch  11  loss =  40.76414132457327
epoch  12  loss =  40.704882410516106
epoch  13  loss =  40.64659347698554
epoch  14  loss =  40.58930107483027
epoch  15  loss =  40.53302803671992
epoch  16  loss =  40.477793544183314
epoch  17  loss =  40.42361322248152
epoch  18  loss =  40.37049925995194
epoch  19  loss =  40.31846054832127
epoch  20  loss =  40.26750284044784
epoch  21  loss =  40.217628921998816
epoch  22  loss =  40.16883879369062
epoch  23  loss =  40.12112986090252
epoch  24  loss =  40.07449712770483
epoch  25  loss =  40.02893339260808
epoch  26  loss =  39.98442944362798
epoch  27  loss =  39.94097425056132
epoch  

Añadiendo subsampling y negative sampling.

In [13]:
import random

def prepare_data_for_training(sentences, w2v, subsample_threshold=1e-5, negative_sampling_table_size=1e6, negative_sampling_power=0.75, negative_samples=5):
    data = {}
    word_counts = {}
    total_words = 0

    for sentence in sentences:
        for word in sentence:
            if word not in data:
                data[word] = 1
            else:
                data[word] += 1
            total_words += 1

    subsampled_data = {}
    for word, freq in data.items():
        prob = 1 - np.sqrt(subsample_threshold / (freq / total_words))
        if random.random() < prob:
            continue
        subsampled_data[word] = freq

    V = len(data)
    data = sorted(list(data.keys()))
    subsampled_vocab = {word: i for i, (word, _) in enumerate(sorted(subsampled_data.items()))}

    sampling_table = []
    sampling_table_weights = []
    for word, freq in subsampled_data.items():
        sampling_table_weights.append(freq ** negative_sampling_power)
    total_weight = sum(sampling_table_weights)
    for i, weight in enumerate(sampling_table_weights):
        prob = weight / total_weight
        sampling_table.extend([i] * int(prob * negative_sampling_table_size))
    sampling_table = np.array(sampling_table)

    for sentence in sentences:
        for i, center_word in enumerate(sentence):
            if center_word not in subsampled_vocab:
                continue
            center_word_index = subsampled_vocab[center_word]
            context = []
            for j in range(i - w2v.window_size, i + w2v.window_size + 1):
                if i != j and 0 <= j < len(sentence) and sentence[j] in subsampled_vocab:
                    context.append(subsampled_vocab[sentence[j]])
            negative_samples_indices = np.random.choice(sampling_table, size=negative_samples)
            w2v.X_train.append(center_word_index)
            w2v.y_train.append(context)
            w2v.y_train.extend(negative_samples_indices)

    w2v.initialize(V, data)

    return w2v.X_train, w2v.y_train


In [16]:
corpus = ""
corpus += "The earth revolves around the sun. The moon revolves around the earth"
epochs = 1000

training_data = preprocessing(corpus)
w2v = word2vec()

prepare_data_for_training(training_data,w2v)
w2v.train(epochs)

print(w2v.predict("around",3))

epoch  1  loss =  0
epoch  2  loss =  0
epoch  3  loss =  0
epoch  4  loss =  0
epoch  5  loss =  0
epoch  6  loss =  0
epoch  7  loss =  0
epoch  8  loss =  0
epoch  9  loss =  0
epoch  10  loss =  0
epoch  11  loss =  0
epoch  12  loss =  0
epoch  13  loss =  0
epoch  14  loss =  0
epoch  15  loss =  0
epoch  16  loss =  0
epoch  17  loss =  0
epoch  18  loss =  0
epoch  19  loss =  0
epoch  20  loss =  0
epoch  21  loss =  0
epoch  22  loss =  0
epoch  23  loss =  0
epoch  24  loss =  0
epoch  25  loss =  0
epoch  26  loss =  0
epoch  27  loss =  0
epoch  28  loss =  0
epoch  29  loss =  0
epoch  30  loss =  0
epoch  31  loss =  0
epoch  32  loss =  0
epoch  33  loss =  0
epoch  34  loss =  0
epoch  35  loss =  0
epoch  36  loss =  0
epoch  37  loss =  0
epoch  38  loss =  0
epoch  39  loss =  0
epoch  40  loss =  0
epoch  41  loss =  0
epoch  42  loss =  0
epoch  43  loss =  0
epoch  44  loss =  0
epoch  45  loss =  0
epoch  46  loss =  0
epoch  47  loss =  0
epoch  48  loss =  0
e

In [32]:
import wikipedia
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

wikipedia.set_lang("en")
article = wikipedia.page("Dragon")
corpus = article.content

tokenized_corpus = [word_tokenize(sentence.lower()) for sentence in corpus.split(".")]

window_sizes = [2, 5, 10]
vector_dimensions = [50, 100, 200]
learning_rates = [0.01, 0.05, 0.1]

for window_size in window_sizes:
    for vector_dimension in vector_dimensions:
        for learning_rate in learning_rates:
            model_name = f"word2vec_w{window_size}_dim{vector_dimension}_lr{learning_rate}"
            print(f"Training {model_name}...")
            model = Word2Vec(sentences=tokenized_corpus,
                             window=window_size,
                             vector_size=vector_dimension,
                             alpha=learning_rate,
                             min_count=1,
                             sg=1)
            model.save(f"{model_name}.model")
            print(f"{model_name} trained and saved.")


Training word2vec_w2_dim50_lr0.01...
word2vec_w2_dim50_lr0.01 trained and saved.
Training word2vec_w2_dim50_lr0.05...
word2vec_w2_dim50_lr0.05 trained and saved.
Training word2vec_w2_dim50_lr0.1...
word2vec_w2_dim50_lr0.1 trained and saved.
Training word2vec_w2_dim100_lr0.01...
word2vec_w2_dim100_lr0.01 trained and saved.
Training word2vec_w2_dim100_lr0.05...
word2vec_w2_dim100_lr0.05 trained and saved.
Training word2vec_w2_dim100_lr0.1...
word2vec_w2_dim100_lr0.1 trained and saved.
Training word2vec_w2_dim200_lr0.01...
word2vec_w2_dim200_lr0.01 trained and saved.
Training word2vec_w2_dim200_lr0.05...
word2vec_w2_dim200_lr0.05 trained and saved.
Training word2vec_w2_dim200_lr0.1...
word2vec_w2_dim200_lr0.1 trained and saved.
Training word2vec_w5_dim50_lr0.01...
word2vec_w5_dim50_lr0.01 trained and saved.
Training word2vec_w5_dim50_lr0.05...
word2vec_w5_dim50_lr0.05 trained and saved.
Training word2vec_w5_dim50_lr0.1...
word2vec_w5_dim50_lr0.1 trained and saved.
Training word2vec_w5_dim

In [34]:
from scipy.stats import spearmanr

models = {}
window_sizes = [2, 5, 10]
vector_dimensions = [50, 100, 200]
learning_rates = [0.01, 0.05, 0.1]

for window_size in window_sizes:
    for vector_dimension in vector_dimensions:
        for learning_rate in learning_rates:
            model_name = f"word2vec_w{window_size}_dim{vector_dimension}_lr{learning_rate}"
            model = Word2Vec.load(f"{model_name}.model")
            models[model_name] = model

word_pairs = [("king", "queen"), ("man", "woman"), ("sun", "moon")]
human_similarities = [4.1, 3.9, 3.8]

model_similarities = {}
for model_name, model in models.items():
    model_similarities[model_name] = []
    for word1, word2 in word_pairs:
        similarity = model.wv.similarity(word1, word2)
        model_similarities[model_name].append(similarity)

correlations = {}
for model_name, similarities in model_similarities.items():
    spearman_corr, _ = spearmanr(human_similarities, similarities)
    correlations[model_name] = spearman_corr

for model_name, correlation in correlations.items():
    print(f"Correlation for {model_name}: {correlation}")


Correlation for word2vec_w2_dim50_lr0.01: 0.5
Correlation for word2vec_w2_dim50_lr0.05: 0.5
Correlation for word2vec_w2_dim50_lr0.1: -0.5
Correlation for word2vec_w2_dim100_lr0.01: 1.0
Correlation for word2vec_w2_dim100_lr0.05: 0.5
Correlation for word2vec_w2_dim100_lr0.1: -0.5
Correlation for word2vec_w2_dim200_lr0.01: 0.5
Correlation for word2vec_w2_dim200_lr0.05: 0.5
Correlation for word2vec_w2_dim200_lr0.1: -0.5
Correlation for word2vec_w5_dim50_lr0.01: 1.0
Correlation for word2vec_w5_dim50_lr0.05: -0.5
Correlation for word2vec_w5_dim50_lr0.1: -1.0
Correlation for word2vec_w5_dim100_lr0.01: 0.5
Correlation for word2vec_w5_dim100_lr0.05: -0.5
Correlation for word2vec_w5_dim100_lr0.1: -1.0
Correlation for word2vec_w5_dim200_lr0.01: 0.5
Correlation for word2vec_w5_dim200_lr0.05: -0.5
Correlation for word2vec_w5_dim200_lr0.1: -1.0
Correlation for word2vec_w10_dim50_lr0.01: 0.5
Correlation for word2vec_w10_dim50_lr0.05: -1.0
Correlation for word2vec_w10_dim50_lr0.1: 0.5
Correlation for 

### Pregunta 2

La factorización de matrices GloVe y PPMI son dos métodos utilizados en el procesamiento del lenguaje natural (NLP) para capturar relaciones semánticas entre palabras a partir de grandes corpus de texto. Ambos métodos se utilizan para generar representaciones vectoriales de palabras, lo que permite que las relaciones semánticas y sintácticas entre palabras se reflejen en el espacio vectorial.

1 . GloVe (Global Vectors for Word Representation)
GloVe es un modelo de aprendizaje no supervisado para obtener representaciones vectoriales de palabras. Fue desarrollado por investigadores de Stanford y combina elementos de dos enfoques principales en NLP: factorización de matrices y modelos basados en ventana de contexto (como word2vec). La idea principal detrás de GloVe es que las co-ocurrencias de palabras en un corpus pueden proporcionar información semántica valiosa.

El modelo GloVe construye una matriz de co-ocurrencia global que tabula cuántas veces cada palabra aparece en el contexto de otras palabras dentro de un corpus. Luego, esta matriz se factoriza para reducir su dimensión, resultando en vectores de palabras más densos. El objetivo de la factorización es mantener la estructura semántica donde la distancia entre dos vectores de palabras refleje la similitud semántica entre las palabras correspondientes.

2 . PPMI (Positive Pointwise Mutual Information)
La PPMI es una técnica que se usa para calcular la asociación entre palabras basada en cuán frecuentemente aparecen juntas en comparación con cuán frecuentemente aparecen por separado. El "Pointwise Mutual Information" (PMI) de dos palabras mide la probabilidad de co-ocurrencia de las palabras en relación con las probabilidades de que cada palabra ocurra por sí sola. Sin embargo, PMI puede tener valores negativos, lo que puede ser problemático en algunos escenarios de modelado.

Para solucionarlo, se utiliza PPMI, donde todos los valores negativos de PMI se reemplazan por cero, enfocándose solo en las asociaciones positivas. En NLP, la PPMI a menudo se usa como una técnica de pre-procesamiento para construir matrices de características que luego pueden ser factorizadas (similar a SVD en GloVe) para obtener representaciones vectoriales de palabras.

Para implementar PPMI, primero construiremos una matriz de co-ocurrencia y luego convertiremos sus valores a PPMI. Usaremos numpy para las operaciones matemáticas y collections para construir la matriz de co-ocurrencia.




In [ ]:
import numpy as np
from collections import defaultdict, Counter
from itertools import product

# Función para construir la matriz de co-ocurrencia
def co_occurrence_matrix(corpus, window_size=2):
    vocab = set(corpus)
    vocab = {word: i for i, word in enumerate(vocab)}
    co_occurrences = defaultdict(Counter)

    for i in range(len(corpus)):
        token = corpus[i]
        left = max(0, i-window_size)
        right = min(len(corpus), i+window_size+1)

        for j in range(left, right):
            if i != j:
                co_occurrences[token][corpus[j]] += 1

    matrix = np.zeros((len(vocab), len(vocab)))

    for token1, neighbors in co_occurrences.items():
        for token2, count in neighbors.items():
            matrix[vocab[token1], vocab[token2]] = count

    return matrix, vocab

# Función para calcular PPMI
def ppmi_matrix(co_matrix, eps=1e-8):
    total_sum = np.sum(co_matrix)
    row_sums = np.sum(co_matrix, axis=1)
    col_sums = np.sum(co_matrix, axis=0)

    ppmi = np.maximum(
        np.log((co_matrix * total_sum) / (row_sums[:, None] * col_sums[None, :] + eps)),
        0
    )
    return ppmi

# Ejemplo de uso
corpus = "the quick brown fox jumps over the lazy dog".split()
co_matrix, vocab = co_occurrence_matrix(corpus, window_size=2)
ppmi = ppmi_matrix(co_matrix)

print(ppmi)


Implementar GloVe desde cero es más complejo debido a la optimización necesaria para ajustar los vectores de palabras. Sin embargo, puedes usar la biblioteca gensim, que tiene una implementación eficiente de GloVe. Utiliza el código realizado en clase.

In [ ]:
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

# Crear modelo Word2Vec con los mismos parámetros que GloVe
modelo = Word2Vec(sentences=[corpus], vector_size=100, window=5, min_count=1, sg=0, workers=4, epochs=10)

# Guardar y cargar el modelo (simulando una carga de GloVe)
model.wv.save_word2vec_format('model.bin')
glove_model = KeyedVectors.load_word2vec_format('model.bin', binary=True)

# Usar el modelo
print(glove_model['fox'])  # Muestra el vector para la palabra "fox"


#### Ejercicios

1. Modifica el tamaño de la ventana de contexto en la función co_occurrence_matrix para diferentes valores (por ejemplo, 1, 3, y 5) y observa cómo cambia la matriz PPMI resultante. Analiza cómo el tamaño de la ventana afecta las relaciones semánticas capturadas (1 punto).
2. Implementa una función que identifique y muestre las palabras con mayor asociación (mayores valores PPMI) para una palabra dada. Utiliza esta función para explorar las relaciones semánticas de varias palabras clave en un corpus más grande (1 punto).
3. Usa la biblioteca gensim para entrenar un modelo GloVe con un corpus más grande (por ejemplo, un conjunto de datos de reseñas de productos o artículos de noticias). Ajusta diferentes hiperparámetros como el tamaño del vector, el tamaño de la ventana, y el número de iteraciones. Evalúa los vectores de palabras resultantes en tareas de analogía y similaridad (1 punto).
4. Realiza una comparación cualitativa y cuantitativa de las representaciones de palabras obtenidas a través de PPMI y GloVe. Considera aspectos como la capacidad de capturar sinónimos, antónimos y relaciones semánticas complejas. Discute en qué casos un método podría ser preferido sobre el otro (1 punto).

In [ ]:
## Tus respuestas

### Pregunta 3

El desarrollo de modelos de redes neuronales recurrentes (RNNs) ha sido fundamental en el avance del procesamiento de secuencias de tiempo y lenguaje natural. Estos modelos son especialmente útiles en tareas como el reconocimiento de voz, la traducción automática y la generación de texto. Sin embargo, las RNNs básicas enfrentan desafíos significativos, como la desaparición y la explosión del gradiente, que obstaculizan su capacidad para aprender dependencias a largo plazo en los datos. Las unidades de memoria de largo y corto plazo (LSTM) y las unidades recurrentes con compuertas (GRU) se desarrollaron como soluciones a estos problemas, mejorando la capacidad de las redes para aprender de datos secuenciales a largo plazo.

Una RNN básica procesa información secuencial mediante la actualización de su estado oculto con cada nuevo elemento de la secuencia. La naturaleza recurrente de estas redes les permite mantener una forma de 'memoria' sobre los elementos anteriores de la secuencia, utilizando la siguiente fórmula básica para actualizar el estado oculto en cada paso de tiempo $t$:

$$
h_t = \sigma(W_{ih} x_t + W_{hh} h_{t-1} + b_h)
$$

Donde $x_t$ es la entrada en el tiempo $t$, $h_t$ es el estado oculto en el tiempo $t$, $W_{ih}$ y $W_{hh}$ son los pesos de entrada y recurrentes, respectivamente, $b_h$ es el término de sesgo, y $\sigma$ es una función de activación no lineal como tanh o ReLU.


El entrenamiento de RNNs implica ajustar estos pesos mediante retropropagación a través del tiempo, lo que puede llevar a dos problemas principales:

1. **Desaparición del gradiente:** Si los gradientes de los pesos son muy pequeños, disminuyen exponencialmente a medida que se propagan hacia atrás a través de cada paso de tiempo. Esto hace que sea difícil para la RNN aprender dependencias a largo plazo, ya que los gradientes se vuelven insignificantes para ajustar los pesos efectivamente en pasos de tiempo anteriores.

2. **Explosión del gradiente:** En contraste, si los gradientes son demasiado grandes, pueden crecer exponencialmente durante la retropropagación, lo que lleva a actualizaciones de peso grandes e inestables, y por ende, a un modelo que diverge y no aprende de manera efectiva.

#### Unidad de memoria de largo y corto plazo (LSTM)

Para abordar estos problemas, se introdujeron las LSTMs, que incorporan un diseño más complejo que permite controlar el flujo de información. Las LSTMs utilizan varias "puertas" para regular tanto el almacenamiento como la eliminación de información en el estado de la celda:

- **Puerta de olvido $(f_t)$** decide qué parte de la información anterior se mantiene:
  $$
  f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)
  $$

- **Puerta de entrada ($i_t$) y candidato de celda ($\tilde{c}_t$)** deciden qué nueva información se añade al estado de la celda:

  $$
  i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)
  $$
  $$
  \tilde{c}_t = \tanh(W_c \cdot [h_{t-1}, x_t] + b_c)
  $$

- **Actualización del estado de la celda ($c_t$)** combina la información antigua y nueva:
  $$
  c_t = f_t \ast c_{t-1} + i_t \ast \tilde{c}_t
  $$

- **Puerta de salida ($o_t$)** y el estado oculto resultante ($h_t$) que determina qué parte del estado de la celda afectará la salida:
  $$
  o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)
  $$
  $$
  h_t = o_t \ast \tanh(c_t)
  $$

#### Unidad recurrente compuerta (GRU)

Las GRUs simplifican la arquitectura de las LSTMs combinando las puertas de entrada y olvido en una sola puerta de actualización y omitiendo el uso de un estado de celda separado:

- **Puerta de actualización ($z_t$)** decide cuánto del estado anterior se debe mantener:
  $$
  z_t = \sigma(W_z \cdot [h_{t-1}, x_t] + b_z)
  $$

- **Puerta de reinicio ($r_t$)** decide cuánto del pasado se debe olvidar antes de calcular el nuevo candidato de estado:
  $$
  r_t = \sigma(W_r \cdot [h_{t-1}, x_t] + b_r)
  $$

- **Candidato de estado oculto ($\tilde{h}_t$)** y la actualización del estado oculto:
  $$
  \tilde{h}_t = \tanh(W_h \cdot [r_t \ast h_{t-1}, x_t] + b_h)
  $$
  
  $$
  h_t = (1 - z_t) \ast h_{t-1} + z_t \ast \tilde{h}_t
  $$


#### Ejercicios

1. ¿Qué papel juegan los reguladores como dropout o L2 regularization específicamente en el contexto de RNNs y LSTM para evitar el sobreajuste en tareas de modelado de lenguaje? (1 punto)
2. Considerando la complejidad computacional de BPTT, ¿cuáles son las limitaciones prácticas cuando se usa con RNNs en secuencias muy largas? ¿Cómo podrías mitigar estos problemas en un entorno de producción? (1 punto)


1. Los reguladores como dropout o L2 regularization tienen un papel muy importante en el contexto de RNNs y LSTM ya que a través de ellos es posible evitar el sobreajuste en tareas de modelado de lenguaje. Mediante estas técnicas uno es capaz de reducir la complejidad de los modelos y evitar que se ajusten demasiado a los datos de entrenamiento, lo cual podría causar que se tenga un rendimiento bastante pobre a la hora de probarlo con datos nuevos.

2. Al usar BPTT con RNNs en secuencias muy largas, hay limitaciones prácticas debido a la complejidad computacional que esta posee. Esto puede resultar en un tiempo de entrenamiento  muy prolongado y un uso excesivo de memoria, ralentizando aún más el proceso. Para mitigar estos problemas en un entorno de producción, se pueden utilizar técnicas de optimización como el truncamiento de BPTT o el uso de técnicas de mini-batch. También se puede considerar el uso de GPUs o el entrenamiento en paralelo para asi tener más formas de acelerar el proceso de entrenamiento.

In [44]:
## Parte 3

import torch
from torch import nn
import torch.nn.functional as F

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.input_to_hidden = nn.Linear(input_size + hidden_size, hidden_size)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = torch.tanh(self.input_to_hidden(combined))
        return hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)


class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size

        # Gates
        self.input_to_inputgate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_forgetgate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_outputgate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_cellgate = nn.Linear(input_size + hidden_size, hidden_size)

    def forward(self, input, hidden, cell):
        combined = torch.cat((input, hidden), 1)

        # Calculate gates
        input_gate = torch.sigmoid(self.input_to_inputgate(combined))
        forget_gate = torch.sigmoid(self.input_to_forgetgate(combined))
        output_gate = torch.sigmoid(self.input_to_outputgate(combined))
        cell_gate = torch.tanh(self.input_to_cellgate(combined))

        # Update cell state
        cell = forget_gate * cell + input_gate * cell_gate
        hidden = output_gate * torch.tanh(cell)

        return hidden, cell

    def initHidden(self):
        return torch.zeros(1, self.hidden_size), torch.zeros(1, self.hidden_size)


class GRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(GRU, self).__init__()
        self.hidden_size = hidden_size
        self.input_to_updategate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_resetgate = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_to_newgate = nn.Linear(input_size + hidden_size, hidden_size)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)

        # Calculate gates
        update_gate = torch.sigmoid(self.input_to_updategate(combined))
        reset_gate = torch.sigmoid(self.input_to_resetgate(combined))
        new_gate = torch.tanh(self.input_to_newgate(combined * reset_gate))

        # Update hidden state
        hidden = update_gate * hidden + (1 - update_gate) * new_gate

        return hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden.store)


Extiende la implementación de LSTM para incluir embeddings de palabras y una capa de clasificación, y entrenar el modelo en una tarea de predicción de la siguiente palabra en secuencias de texto (3 puntos).

- Agrega una capa de embedding al modelo LSTM para procesar entradas de texto.
- Incluye una capa de salida que mapee el estado oculto a las predicciones de palabras.
- Implementa una función de pérdida adecuada para la clasificación de palabras.
- Preprocesa  un corpus de texto grande (utiliza los datos dados en clase por ejemplo) para convertir texto a índices utilizando un vocabulario predefinido.
- Genera datos de entrenamiento como pares de secuencias de entrada y palabras objetivo.

In [41]:
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [45]:
class LSTMWithEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
        super(LSTMWithEmbedding, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = LSTM(embedding_dim, hidden_size)

        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden, cell):
        embedded = self.embedding(input)
        lstm_out, (hidden, cell) = self.lstm(embedded, hidden, cell)
        output = self.output_layer(lstm_out)
        return output, hidden, cell

    def initHidden(self):
        return torch.zeros(1, self.hidden_size), torch.zeros(1, self.hidden_size)


In [38]:
class TextDataset(Dataset):
    def __init__(self, text_data, word_to_index, context_size):
        self.text_data = text_data
        self.word_to_index = word_to_index
        self.context_size = context_size
        self.data = self.generate_data()

    def generate_data(self):
        data = []
        for i in range(len(self.text_data) - self.context_size):
            context = [self.word_to_index[word] for word in self.text_data[i:i+self.context_size]]
            target = self.word_to_index[self.text_data[i+self.context_size]]
            data.append((context, target))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context, target = self.data[idx]
        return torch.tensor(context), torch.tensor(target)

In [39]:
def preprocess_text(text_corpus, vocab):
    word_to_index = {word: i for i, word in enumerate(vocab)}
    indexed_corpus = [word_to_index[word] for word in text_corpus if word in vocab]
    return indexed_corpus, word_to_index

def generate_training_data(indexed_corpus, context_size):
    training_data = []
    for i in range(len(indexed_corpus) - context_size):
        context = indexed_corpus[i:i+context_size]
        target = indexed_corpus[i+context_size]
        training_data.append((context, target))
    return training_data

loss_function = nn.CrossEntropyLoss()

Realiza un análisis de sensibilidad de los hiperparámetros en modelos LSTM y GRU para entender su impacto en la capacidad de aprendizaje de dependencias a largo plazo en textos (3 puntos)

- Selecciona un corpus de texto y prepara datos para el entrenamiento de modelos de lenguaje basados en LSTM y GRU.
- Experimenta con diferentes valores para los hiperparámetros como el tamaño de las puertas, la tasa de aprendizaje, el tamaño del estado oculto y la longitud de BPTT.
- Utiliza técnicas como validación cruzada para evaluar el impacto de estos cambios en la precisión del modelo y en su capacidad para generar texto coherente.
- Analiza cómo la modificación de los parámetros de las puertas y la longitud de BPTT afecta la estabilidad del entrenamiento y la convergencia del modelo.

In [ ]:
## Tus respuestas

### Pregunta 4
El script proporcionado es un ejemplo completo de cómo implementar un modelo de red neuronal recurrente (RNN) utilizando PyTorch para generar texto de manera automática.

In [ ]:
# Importación de librerías necesarias para trabajar con tensores y redes neuronales.
import torch
from torch import nn
import numpy as np

# Datos de entrada: una lista de frases.
text = ['hey how are you','good i am fine','have a nice day']

# Creación de un conjunto de caracteres únicos presentes en las frases.
chars = set(''.join(text))
# Creación de un diccionario que mapea cada caracter a un índice único.
int2char = dict(enumerate(chars))
# Creación de un diccionario inverso que mapea cada índice a su caracter correspondiente.
char2int = {char: ind for ind, char in int2char.items()}

# Determinación de la longitud máxima de las frases para normalizar la longitud de todas.
maxlen = len(max(text, key=len))
print("La longitud mayor tiene {} caracteres".format(maxlen))

# Añadir espacios a las frases más cortas para igualar la longitud máxima.
for i in range(len(text)):
  while len(text[i])<maxlen:
    text[i] += ' '

# Inicialización de listas para secuencias de entrada y objetivo.
input_seq = []
target_seq = []

# Creación de secuencias de entrada y objetivo.
for i in range(len(text)):
    input_seq.append(text[i][:-1])
    target_seq.append(text[i][1:])
    print("Secuencia entrada: {}\nSecuencia objetivo: {}".format(input_seq[i], target_seq[i]))

# Conversión de caracteres a índices para procesamiento numérico.
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

# Definición de tamaños para la codificación one-hot.
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

# Función para codificar las secuencias en formato one-hot.
def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    for i in range(batch_size):
        for u in range(seq_len):
          features[i, u, sequence[i][u]] = 1
    return features

# Aplicación de la codificación one-hot a las secuencias de entrada.
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Forma de entrada: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))

# Conversión de las secuencias de entrada a tensores de PyTorch.
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

# Chequeo de disponibilidad de GPU y selección del dispositivo (GPU o CPU).
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU es disponible")
else:
    device = torch.device("cpu")
    print("GPU no disponible, CPU es usada")

# Definición de la clase del modelo RNN.
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Capa RNN que toma entradas y retorna la salida y un estado oculto.
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)
        # Capa lineal que procesa la salida del RNN.
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        # Inicialización del estado oculto a cero.
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        return hidden

# Instancia del modelo con parámetros específicos.
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
model.to(device)

# Definición de hiperparámetros para el entrenamiento.
n_epochs = 100
lr=0.01

# Configuración de la función de pérdida y el optimizador.
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Bucle de entrenamiento del modelo.
for epoch in range(1, n_epochs + 1):
  optimizer.zero_grad()
  input_seq = input_seq.to(device)
  output, hidden = model(input_seq)
  loss = criterion(output, target_seq.view(-1).long())
  loss.backward() # Realización de backpropagation y cálculo de gradientes.
  optimizer.step() # Actualización de los pesos del modelo.

  if epoch%10 == 0:
    print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
    print("Loss: {:.4f}".format(loss.item()))

# Funciones para predicción y generación de texto basadas en el modelo entrenado.
def predict(model, character):
  character = np.array([[char2int[c] for c in character]])
  character = one_hot_encode(character, dict_size, character.shape[1], 1)
  character = torch.from_numpy(character)
  character.to(device)

  out, hidden = model(character)

  prob = nn.functional.softmax(out[-1], dim=0).data
  char_ind = torch.max(prob, dim=0)[1].item()

  return int2char[char_ind], hidden

def sample(model, out_len, start='hey'):
  model.eval()
  start = start.lower()
  chars = [ch for ch in start]
  size = out_len - len(chars)
  for ii in range(size):
    char, h = predict(model, chars)
    chars.append(char)

  return ''.join(chars)

# Ejemplo de uso de la función de generación de texto.
sample(model, 15, 'good')


#### Ejercicios:

1. Modifica el modelo existente para que funcione como un autoencoder. Esto implica que el modelo debe aprender a codificar una secuencia de entrada en un vector de características (estado oculto) y luego decodificar ese vector de vuelta a la secuencia original (1.5 puntos).
     - Implementa las capas de codificación y decodificación dentro del mismo modelo.
     - Experimenta  con diferentes estructuras como LSTM para mejorar la retención de información.
     - Mide la calidad de la reconstrucción del texto y la eficiencia de compresión.

2. Utiliza el modelo RNN actual y modifícalo para introducir secuencias más largas. Monitoriza los gradientes durante el entrenamiento para detectar signos de desaparición o explosión. (1.5 puntos)
    - Implementa el  clipping de gradiente para prevenir la explosión del gradiente.
    - Reemplaza la RNN por LSTM para abordar la desaparición del gradiente.
    - Utiliza técnicas de visualización para observar la magnitud de los gradientes a lo largo de varias épocas.
3. Implementa el dropout en las capas recurrentes y comparar los resultados. (1 punto)

    - Ajusta el parámetro de weight decay en el optimizador y observar el efecto sobre el overfitting.
    - Aplica early stopping basado en la validación del loss para detener el entrenamiento antes de que el modelo comience a sobreajustarse.


In [ ]:
## Tus respuestas